# Sondera Harness Quickstart

**Run your first policy evaluation in under a minute.**

The more you trust an agent, the more you can let it do. But trust requires guarantees. This notebook shows how Sondera Harness lets you define what an agent is allowed to do, in code. The agent stays within those limits, or it doesn't act.

We'll build a policy for a coding agent that:
- **Allows** reading any file
- **Allows** writing files, *except* `.env`, credentials, and secrets
- **Allows** bash commands, *except* destructive ones like `rm -rf /`
- **Allows** web access, *except* untrusted domains like pastebin

*This is a simulation: we evaluate what would be allowed, but nothing actually executes.*

## Install

In [ ]:
%%capture
%pip install sondera-harness

## Define the Agent

An agent is defined by its tools. Each tool has a name, description, and parameters that policies can reference.

In [ ]:
import json

from cedar import PolicySet
from sondera import (
    Agent,
    CedarPolicyHarness,
    Decision,
    Parameter,
    Role,
    Stage,
    Tool,
    ToolRequestContent,
)
from sondera.harness.cedar.schema import agent_to_cedar_schema

# Define the tools your agent can use
# parameters_json_schema is required for Cedar to access tool arguments

read_tool = Tool(
    name="Read",
    description="Read the contents of a file from the filesystem",
    parameters=[
        Parameter(
            name="file_path", description="The absolute path to the file", type="string"
        ),
    ],
    parameters_json_schema=json.dumps(
        {
            "type": "object",
            "properties": {"file_path": {"type": "string"}},
            "required": ["file_path"],
        }
    ),
)

write_tool = Tool(
    name="Write",
    description="Write or overwrite a file with new content",
    parameters=[
        Parameter(
            name="file_path", description="The absolute path to the file", type="string"
        ),
        Parameter(name="content", description="The content to write", type="string"),
    ],
    parameters_json_schema=json.dumps(
        {
            "type": "object",
            "properties": {
                "file_path": {"type": "string"},
                "content": {"type": "string"},
            },
            "required": ["file_path", "content"],
        }
    ),
)

edit_tool = Tool(
    name="Edit",
    description="Make targeted edits to a file by replacing exact string matches",
    parameters=[
        Parameter(
            name="file_path", description="The absolute path to the file", type="string"
        ),
        Parameter(name="old_string", description="The text to find", type="string"),
        Parameter(
            name="new_string", description="The text to replace with", type="string"
        ),
    ],
    parameters_json_schema=json.dumps(
        {
            "type": "object",
            "properties": {
                "file_path": {"type": "string"},
                "old_string": {"type": "string"},
                "new_string": {"type": "string"},
            },
            "required": ["file_path", "old_string", "new_string"],
        }
    ),
)

glob_tool = Tool(
    name="Glob",
    description="Find files matching a glob pattern",
    parameters=[
        Parameter(
            name="pattern",
            description="The glob pattern (e.g., '**/*.py')",
            type="string",
        ),
    ],
    parameters_json_schema=json.dumps(
        {
            "type": "object",
            "properties": {"pattern": {"type": "string"}},
            "required": ["pattern"],
        }
    ),
)

bash_tool = Tool(
    name="Bash",
    description="Execute a shell command",
    parameters=[
        Parameter(name="command", description="The command to execute", type="string"),
    ],
    parameters_json_schema=json.dumps(
        {
            "type": "object",
            "properties": {"command": {"type": "string"}},
            "required": ["command"],
        }
    ),
)

web_fetch_tool = Tool(
    name="WebFetch",
    description="Fetch content from a URL",
    parameters=[
        Parameter(name="url", description="The URL to fetch", type="string"),
        Parameter(name="prompt", description="What to extract", type="string"),
    ],
    parameters_json_schema=json.dumps(
        {
            "type": "object",
            "properties": {"url": {"type": "string"}, "prompt": {"type": "string"}},
            "required": ["url", "prompt"],
        }
    ),
)

agent = Agent(
    id="coding_agent",
    provider_id="custom",
    name="Coding Agent",
    description="A coding assistant with file operations, shell commands, and web access.",
    instruction="You are a coding assistant with access to file operations, shell commands, and web access.",
    tools=[read_tool, write_tool, edit_tool, glob_tool, bash_tool, web_fetch_tool],
)

print(
    f"Agent '{agent.name}' with {len(agent.tools)} tools: {[t.name for t in agent.tools]}"
)

## Define the Policy

The policy is written in [Cedar](https://www.cedarpolicy.com/), a language designed for authorization. The pattern: **permit by default, then forbid specific things**. `forbid` always wins over `permit`.

The namespace `Coding_Agent` is derived from the agent name (spaces become underscores).

In [ ]:
policy_set = PolicySet("""
// Allow all read operations - reading is generally safe
@id("allow-read")
permit(principal, action == Coding_Agent::Action::"Read", resource);

// Allow file pattern searches
@id("allow-glob")
permit(principal, action == Coding_Agent::Action::"Glob", resource);

// Allow write operations by default (restricted below)
@id("allow-write")
permit(principal, action == Coding_Agent::Action::"Write", resource);

// Allow edit operations by default (restricted below)
@id("allow-edit")
permit(principal, action == Coding_Agent::Action::"Edit", resource);

// Forbid writing to sensitive files
@id("forbid-sensitive-write")
forbid(principal, action == Coding_Agent::Action::"Write", resource)
when {
  context has parameters &&
  (context.parameters.file_path like "*.env*" ||
   context.parameters.file_path like "*credentials*" ||
   context.parameters.file_path like "*secrets*")
};

// Forbid editing sensitive files
@id("forbid-sensitive-edit")
forbid(principal, action == Coding_Agent::Action::"Edit", resource)
when {
  context has parameters &&
  (context.parameters.file_path like "*.env*" ||
   context.parameters.file_path like "*id_rsa*" ||
   context.parameters.file_path like "*.pem*")
};

// Allow bash commands by default (restricted below)
@id("allow-bash")
permit(principal, action == Coding_Agent::Action::"Bash", resource);

// Forbid dangerous bash commands
@id("forbid-dangerous-bash")
forbid(principal, action == Coding_Agent::Action::"Bash", resource)
when {
  context has parameters &&
  (context.parameters.command like "*rm -rf /*" ||
   context.parameters.command like "*mkfs*" ||
   context.parameters.command like "*dd if=/dev/zero*")
};

// Allow web fetches by default (restricted below)
@id("allow-web-fetch")
permit(principal, action == Coding_Agent::Action::"WebFetch", resource);

// Forbid fetching from untrusted domains
@id("forbid-untrusted-fetch")
forbid(principal, action == Coding_Agent::Action::"WebFetch", resource)
when {
  context has parameters &&
  (context.parameters.url like "*pastebin*" ||
   context.parameters.url like "*raw.githubusercontent.com*")
};
""")

print(f"Policy loaded with {len(policy_set)} rules")

## Initialize the Harness

Sondera generates a Cedar schema from your tools. The harness evaluates each action against your policy using `adjudicate()`.

In [ ]:
schema = agent_to_cedar_schema(agent)  # Converts tools to Cedar types
harness = CedarPolicyHarness(policy_set=policy_set, schema=schema)
await harness.initialize(agent=agent)

print("Harness initialized and ready")

## Run It

Now let's test what the agent is allowed to do. We call `adjudicate()` for each action, and the harness returns `ALLOW` or `DENY`.

**Nothing actually executes.** We're just asking: "Would this be allowed?"

In [ ]:
# ANSI color codes for terminal output
GREEN = "\033[92m"
RED = "\033[91m"
RESET = "\033[0m"


def check(description: str, result, expected: Decision):
    """Check result matches expected and print colored status."""
    if result.decision == Decision.ALLOW:
        color = GREEN
        status = "ALLOW"
    else:
        color = RED
        status = "DENY"

    icon = "\u2705" if result.decision == expected else "\u274c"
    print(f"{icon} {description} ({color}{status}{RESET})")
    assert result.decision == expected, f"Expected {expected}, got {result.decision}"


# =============================================================================
# SAFE OPERATIONS - these should be ALLOWED by the policy
# =============================================================================

# Reading is always allowed (allow-read)
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(tool_id="Read", args={"file_path": "/code/main.py"}),
)
check("Reading a file", result, Decision.ALLOW)

# Writing to a normal file is allowed (allow-write, no forbid match)
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="Write",
        args={"file_path": "/code/test.py", "content": "print('hello')"},
    ),
)
check("Writing to test.py", result, Decision.ALLOW)

# Safe bash commands are allowed (allow-bash, no forbid match)
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(tool_id="Bash", args={"command": "git status"}),
)
check("Running 'git status'", result, Decision.ALLOW)

# Glob searches are always allowed (allow-glob)
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(tool_id="Glob", args={"pattern": "**/*.py"}),
)
check("Glob search for Python files", result, Decision.ALLOW)

# =============================================================================
# DANGEROUS OPERATIONS - these should be DENIED by forbid rules
# =============================================================================

# Blocked by forbid-sensitive-write (matches *.env*)
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="Write", args={"file_path": "/code/.env", "content": "API_KEY=secret"}
    ),
)
check("Writing to .env", result, Decision.DENY)

# Blocked by forbid-dangerous-bash (matches *rm -rf /*)
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(tool_id="Bash", args={"command": "rm -rf /"}),
)
check("Running 'rm -rf /'", result, Decision.DENY)

# Blocked by forbid-sensitive-edit (matches *id_rsa*)
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="Edit",
        args={
            "file_path": "/home/user/.ssh/id_rsa",
            "old_string": "old",
            "new_string": "new",
        },
    ),
)
check("Editing SSH key", result, Decision.DENY)

# Blocked by forbid-untrusted-fetch (matches *pastebin*)
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="WebFetch",
        args={"url": "https://pastebin.com/raw/abc123", "prompt": "Get content"},
    ),
)
check("Fetching from pastebin", result, Decision.DENY)

print(f"\n{GREEN}{'=' * 50}")
print(f"All tests passed! Safe operations allowed, dangerous ones blocked.{RESET}")

## Try It Yourself

Experiment with the policy. Go back to the **Define the Policy** cell, make a change, and re-run:

- **Block a new command**: Add `context.parameters.command like "*curl*"` to `forbid-dangerous-bash`
- **Allow a blocked path**: Remove `context.parameters.file_path like "*credentials*"` from `forbid-sensitive-write`
- **Change a pattern**: Try `*password*` or `*config*` in a `like` clause

Or just modify the test cases below to try different file paths, commands, or URLs:

In [ ]:
# Your experiments here! Try changing paths, commands, or URLs.


def show(description: str, result):
    """Print colored result."""
    color = GREEN if result.decision == Decision.ALLOW else RED
    print(f"{description}: {color}{result.decision.name}{RESET}")


# This should be DENIED (matches *credentials*)
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="Write", args={"file_path": "/code/credentials.json", "content": "{}"}
    ),
)
show("Writing to credentials.json", result)

# This should be DENIED (matches *secrets*)
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="Write",
        args={"file_path": "/app/secrets/api_key.txt", "content": "sk-123"},
    ),
)
show("Writing to secrets folder", result)

# This should be ALLOWED (not in any forbid pattern)
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="Bash", args={"command": "curl https://api.example.com"}
    ),
)
show("Running curl command", result)

# This should be DENIED (matches *raw.githubusercontent.com*)
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="WebFetch",
        args={
            "url": "https://raw.githubusercontent.com/user/repo/main/script.sh",
            "prompt": "Get script",
        },
    ),
)
show("Fetching from raw GitHub", result)

# This should be ALLOWED (safe URL)
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="WebFetch",
        args={"url": "https://docs.python.org/3/", "prompt": "Get docs"},
    ),
)
show("Fetching Python docs", result)

## Next Steps

- [Integrations](https://docs.sondera.ai/integrations/) - Add Sondera to LangGraph, ADK, or Strands
- [Writing Policies](https://docs.sondera.ai/writing-policies/) - Cedar syntax and common patterns
- [Core Concepts](https://docs.sondera.ai/concepts/) - Understand stages, decisions, and trajectories